In [ ]:
# Import bibliotek
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from time import sleep

In [ ]:
# funkcja wykorzystana w kodach:

def zmiana_na_float (cecha):
    """ 
    Funkcja do zmiany danych poszczegolnych cech na float z odcięciem jednostki.
      
    Parametry:
    ----------
    cecha (str): Liczba podana jako ciąg znaków 
  
    Wynik:
    ------
    float: Ciąg znaków zamieniony na float.
    
    Przykład:
    ---------
    zmiana_na_float('50,2 m2')->50.2
      
    """
    
    return float(cecha.rsplit(' ',1)[0].replace(' ','').replace(',','.'))     #rsplit-rozdzielam po spacji, ale tylko jednej spacji licząc od końca. Jeśli w stringu będzie kilka spacji, to podzieli tylko po tej ostatniej   

## Kod do pobrania szczegółów ogłoszeń (widocznych z poziomu strony głównej)

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam jedynie dane z pierwszej strony zwróconej przez Otodom. Są to dane widoczne z poziomu tej strony ( bez zagłębiania się w szczegóły, które można zobaczyć tylko po wejściu w konkretne oferty)
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom, jak również szczegółowe dane widoczne tylko po wejściu w konkretną ofertę 


In [ ]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strony)
zawartosc = requests.get('https://www.otodom.pl/sprzedaz/mieszkanie/poznan/?search%5Bfilter_float_m%3Ato%5D=30&search%5Bfilter_enum_market%5D%5B0%5D=secondary&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1')

In [ ]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [ ]:
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert


for item in wszystkie_oferty_items: # dla każdej cechy, którą chcę wydobyć, szukam jej klasy na stronie Otodom
    naglowek=item.find(class_='text-nowrap hidden-xs').get_text()
    liczba_pokoi=item.find(class_="offer-item-rooms hidden-xs").get_text()
    powierzchnia=item.find(class_="hidden-xs offer-item-area").get_text()
    cena=item.find(class_="offer-item-price").get_text()
    cena_zlm2=item.find(class_="hidden-xs offer-item-price-per-m").get_text()
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        
    # poniższe cechy zamieniam na float wg.wczesniej napisanej funkcji
    liczba_pokoi=zmiana_na_float(liczba_pokoi)
    powierzchnia=zmiana_na_float(powierzchnia)
    cena=zmiana_na_float(cena.strip())
    cena_zlm2 =zmiana_na_float(cena_zlm2)

    print('tytuł: ', naglowek)
    print('liczba_pokoi: ', liczba_pokoi)
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zl_m2): ', cena_zlm2)
    print(adres_www, '\n')
    
    f.writerow([naglowek, liczba_pokoi,powierzchnia, cena, cena_zlm2, adres_www ]) ## zapisanie do csv

In [ ]:
## zapisanie do CSV
f = csv.writer(open('oferty_mieszkania.csv', 'w'))
f.writerow(['tytuł', 'liczba_pokoi', 'powierzchnia','cena', 'cena_zlm2', 'adres_www' ])

## Kod do pobrania szczegółów zawartych wewnątrz konkretnych ogłoszeń

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, powierzchni, rodzaju nieruchomości - w tym wypadku mieszkania, itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam dane z pierwszej strony zwróconej przez Otodom, przy czym są to dane szczegółowe widoczne po otwarciu konkretnych oferT
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom

In [ ]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strona)
zawartosc = requests.get('https://www.otodom.pl/sprzedaz/mieszkanie/poznan/?search%5Bfilter_float_m%3Ato%5D=30&search%5Bfilter_enum_market%5D%5B0%5D=secondary&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1')

In [ ]:
zawartosc.status_code

In [ ]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [ ]:
# kod do zaczytania szczegółów zawartych w ofertach (przy przeglądaniu strony www widoczne są tylko po wejściu w daną ofertę)
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

for item in wszystkie_oferty_items:
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        zawartosc2=requests.get(adres_www)
        soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
        szczegoly=soup2.find_all(class_='article-offer')
        for item in szczegoly:       
            adres=item.find(class_='address-lane').get_text()
            powierzchnia=item.find(class_='big').get_text()
            cena=item.find(class_="box-price-value").get_text()
            try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                pietro=item.find(class_='param_floor_no').get_text()
            except AttributeError:
                pietro='brak informacji'
            liczba_pokoi=item.find(class_='room-lane').get_text()
            opis=item.find(itemprop='description').get_text()


        powierzchnia=zmiana_na_float(powierzchnia)   
        cena=zmiana_na_float(cena.strip())
        cena_zl_m2=round(cena/powierzchnia,2)
        liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]



    print('adres: ', adres.strip())
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zł/m2): ', cena_zl_m2)
    print('pietro: ', pietro)
    print('liczba_pokoi: ', liczba_pokoi[0])
    print('opis: ', opis)
    print(adres_www, '\n')

    f.writerow([adres, powierzchnia,cena, cena_zl_m2, pietro, liczba_pokoi[0], opis ]) ## zapisanie do csv
        

In [ ]:
f = csv.writer(open('oferty_mieszkania.csv', 'w'))
f.writerow(['adres', 'powierzchnia (m2)', 'cena (zł)','Cena (zł/m2)','pietro', 'liczba_pokoi','opis' ])

## Kod do pobrania ofert z kilku stron 

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w poprzednich kodach czytałam jedynie dane widoczne na pierwszej stronie zwrócoej przez Otodom, teraz zaczytam dane z wielu stron. Zrobię to na 2 sposoby:
    - w pierwszym kodzie użytkownik sam będzie podawał, z ilu stron chce uzyskać dane ( np. Otodom zwróci 10 stron, a użytkownik uzna, że chce dane tylko z 2 stron). Poza tym użytkownik będzie mógł wybrać, z których stron chce otrzymać dane, wpisując linki do tych stron
    - w drugim kodzie automatycznie zaczytam dane z wszystkich stron zwróconych przez Otodom
- dodatkowo uporządkuję pobrane dane wyorzystując bibliotekę Pandas (do csv będą eksportowane uporządkowane dane)


### Kod 1- użytkownik podaje,z których stron chce pobrać dane

In [ ]:
strony=[]
for strona in range(0,int(input("Podaj, ile stron chcesz przejrzeć: "))):
    strona=input(str("Wpisz pełen adres strony- cały link: "))
    strony.append(strona)
    print('strony: ', strony)

In [ ]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
opisy=[]
    
for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                    pietro=item.find(class_='param_floor_no').get_text()
                except AttributeError:
                    pietro='brak informacji'
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                sleep(1)

                powierzchnia=zmiana_na_float(powierzchnia)   
                cena=zmiana_na_float(cena.strip())
                cena_zl_m2=round(cena/powierzchnia,2)
                liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]


                print('adres: ', adres.strip())# strip rozdziela ciąg znaków i zwraca listę 
                print('powierzchnia (m2): ', powierzchnia)
                print('cena (zł): ', cena)
                print('cena (zł/m2): ', cena_zl_m2)
                print('pietro: ', pietro)
                print('liczba_pokoi: ', liczba_pokoi[0])
                print('opis: ', opis)
                print(adres_www, '\n')

                # zapisuję wyciągnięte dane do takiej postaci, abym mogła je dalej zapisać w Pandasie
                adresy.append(adres.strip())
                powierzchnie.append(powierzchnia)
                ceny.append(cena)
                ceny_zlm2.append(cena_zl_m2)
                pietra.append(pietro)
                pokoje.append(liczba_pokoi[0])
                opisy.append(opis)


### Obróbka  danych - Pandas

In [23]:
# data frame 'oferty' zawiera szczegóły pobrane ze strony Otodom. Dalej będę edytować dane w tabeli, aby były one przedstawione w bardziej przejrzysty sposob
oferty=pd.DataFrame({'adres':adresy,
                               'powierzchnia (m2)':powierzchnie,
                                'cena (zł)':ceny,
                                'Cena (zł/m2)':ceny_zlm2,
                                'pietro':pietra, 
                                'liczba_pokoi':pokoje,
                                'opis': opisy})

In [24]:
oferty.head()

,adres,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",28.80,184900.0,6420.14,Piętro 4 (z 5),1,\nIDEALNA OFERTA POD INWESTYCJĘ. KAMIENICA W T...
1,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",29.21,216154.0,7400.00,Piętro 4 (z 4),2,\nKamienica o charakterze hotelowym . Mieszkan...
2,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",27.00,229000.0,8481.48,Piętro 4 (z 4),1,"\nWitam, mam na imię Agata i w biurze nierucho..."
3,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",27.00,199000.0,7370.37,Piętro parter (z 4),1,"\nWitam, mam na imię Agata i w biurze nierucho..."
4,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",27.00,215000.0,7962.96,Piętro 4 (z 4),1,\nKupując z pośrednikiem - kupujesz bezpieczni...


In [25]:
# w kolumnie 'adres' mam string z danymi typu:rodzaj nieruchomosci, wojewodztwo, miasto, dzielnica itd.
# rozdzielam kolumnę 'adres' tak, aby każdy rodzaj danych znalazł się  w oddzielnej kolumnie. Jednocześnie zachowuję dane z pozostałych kolumn
mieszkania_oferty = pd.concat([oferty['adres'].apply(lambda x: pd.Series(str(x).split(','))),\
                     oferty['powierzchnia (m2)'],\
                     oferty['cena (zł)'],\
                     oferty['Cena (zł/m2)'],\
                     oferty['pietro'],\
                     oferty['liczba_pokoi'],\
                     oferty['opis']                
                    ],axis=1,ignore_index=True)
nazwy_kolumn = ['nieruchomosc', 'wojewodztwo', 'miasto', 'dzielnica', 'ulica', 'reszta_inf', 'powierzchnia (m2)','cena (zł)', 'Cena (zł/m2)', 'pietro', 'liczba_pokoi', 'opis']
mieszkania_oferty.columns=nazwy_kolumn

In [26]:
# usuwam kolumnę 'reszta_inf', ponieważ powtarzają się w niej dane zawarte w poprzedzających kolumnach
mieszkania_oferty.drop(['reszta_inf'], axis=1, inplace=True)

In [27]:
#usuwam wyraz'Piętro' z kolumny 'pietro'
mieszkania_oferty['pietro']=mieszkania_oferty['pietro'].apply(lambda x: x.replace('Piętro',''))

In [28]:
# usuwam '\nPoznań' z kolumny 'ulica'
mieszkania_oferty['ulica']=mieszkania_oferty['ulica'].apply(lambda x: x.replace('\nPoznań',''))

In [29]:
# usuwam '\nPoznań' z kolumny 'dzielinca'
mieszkania_oferty['dzielnica']=mieszkania_oferty['dzielnica'].apply(lambda x: x.replace('\nPoznań',''))

In [34]:
mieszkania_oferty.drop_duplicates(subset=['powierzchnia (m2)', 'cena (zł)','pietro','liczba_pokoi' ]).head(6)

,nieruchomosc,wojewodztwo,miasto,dzielnica,ulica,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Głogowska,28.80,184900.0,6420.14,4 (z 5),1,\nIDEALNA OFERTA POD INWESTYCJĘ. KAMIENICA W T...
1,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Łazarz,Gąsiorowskich,29.21,216154.0,7400.00,4 (z 4),2,\nKamienica o charakterze hotelowym . Mieszkan...
2,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Swoboda,27.00,229000.0,8481.48,4 (z 4),1,"\nWitam, mam na imię Agata i w biurze nierucho..."
3,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Marcelińska,27.00,199000.0,7370.37,parter (z 4),1,"\nWitam, mam na imię Agata i w biurze nierucho..."
4,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Grochowska,27.00,215000.0,7962.96,4 (z 4),1,\nKupując z pośrednikiem - kupujesz bezpieczni...
5,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Łazarz,Łukaszewicza,29.68,270000.0,9097.04,brak informacji,1,\nLOKAL MIESZKALNY NA SPRZEDAŻ Polecam do sprz...


In [35]:
mieszkania_oferty.to_csv(r'/home/kinga/python/Web_scrapping/mieszkania_oferty.csv')

### Kod 2 - automatycznie  pobiera dane ze wszystkich stron zwróconych przez Otodom

In [ ]:

strona='https://www.otodom.pl/sprzedaz/mieszkanie/poznan/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bfilter_enum_market%5D%5B0%5D=secondary&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=87&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1'
strony=[strona]
page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')

cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager') # w tym jest klasa 'page-counter' i 'page_next'

przycisk_przewijania_stron=cala_strona.find_all(class_='pager-next')


for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_str=item.find(class_='current').get_text()
    print (liczba_str)




In [ ]:
i=1
while i in range(0,int(liczba_str)):
        i+=1
    
        strony.append(strona +'&page=' + str(i))  # każda kolejna strona ma końcówkę '&page=(nr_str)' w stosunku do strony pierwszej, poczynając od '&page=2' 

In [ ]:
print ('strony: ', strony)

In [ ]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
opisy=[]
    
for item in strony:
    page = requests.get(item)
    soup=BeautifulSoup(page.content, 'html.parser')
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram linka do każdej oferty
            page2=requests.get(adres_www)
            soup2=BeautifulSoup(page2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                    pietro=item.find(class_='param_floor_no').get_text()
                except AttributeError:
                    pietro='brak informacji'
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                sleep(1)

                powierzchnia=zmiana_na_float(powierzchnia)   
                cena=zmiana_na_float(cena.strip())
                cena_zl_m2=round(cena/powierzchnia,2)
                liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]


                print('adres: ', adres.strip())
                print('powierzchnia (m2): ', powierzchnia)
                print('cena (zł): ', cena)
                print('cena (zł/m2): ', cena_zl_m2)
                print('pietro: ', pietro)
                print('liczba_pokoi: ', liczba_pokoi[0])
                print('opis: ', opis)
                print(adres_www, '\n')

                # zapisuję pobrane dane do takiej postaci, abym mogła je dalej przedstawić w Pandasie
                adresy.append(adres.strip())
                powierzchnie.append(powierzchnia)
                ceny.append(cena)
                ceny_zlm2.append(cena_zl_m2)
                pietra.append(pietro)
                pokoje.append(liczba_pokoi[0])
                opisy.append(opis)


In [ ]:
oferty_wszystkie_str=pd.DataFrame({'adres':adresy,
                               'powierzchnia (m2)':powierzchnie,
                                'cena (zł)':ceny,
                                'Cena (zł/m2)':ceny_zlm2,
                                'pietro':pietra, 
                                'liczba_pokoi':pokoje,
                                'opis': opisy})

In [ ]:
oferty_wszystkie_str.to_csv(r'/home/kinga/python/web_scrapping/oferty_wszystkie.csv')